In [7]:
import pandas as pd
import numpy as np
import os
import re
import math
import codecs
from IPython.core.display import display

### Info

This code was developed by Russell Pekala in September 2017 to clean and package raw scraped data from the Harvard Q Guide.  It is in no way perfect - it takes only a minute to completely run so performance optimization was not a priority.  It is supposed to be clear and easy to update/customize if we (or someone else) scrapes the Q-Guide again.

# **Instructions:** 
* Execute the cells in order. 

## Constants

In [21]:
# Columns we won't analyze
drop_cols = ['view_actual_comments', 'view_comments-href', 'view_comments', 
             'class-href', 'view_instructor', 'view_instructor-href', 'exists2instructors?',
             'view_actual_comments-href']

# Make up for gaps in our scraping convention by renaming.
rename_cols = {
    # Main cols.  Python thinks the word class means something different 
    'all': 'all_course',
    'class': 'course',
    
    # Cols related to the course overall
    'overall_img-src': 'course_img1', # From div 1
    'materials_img-src': 'course_img2',
    'assignments_img-src': 'course_img3',
    'feedback_img-src': 'course_img4',
    'section_img-src': 'course_img5', # Still from div 1
    'workload_img-src': 'course_img6', # From div 2
    'recommend_img-src': 'course_img7', # From div 3
    'money_img-src': 'course_img8', # From div 4
    
    # Not sure how this changes by year but it should be generalized anyway
    'elective_img-src': 'reason1',
    'concentration_img-src': 'reason2', 
    'secondary_img-src': 'reason3', 
    'gened_img-src': 'reason4', 
    'expos_img-src': 'reason5', 
    'foreign_img-src': 'reason6', 
    'premed_img-src': 'reason7',
    
    # For the professor
    'overall_img_instructor-src': 'instructor_img1',
    'lectures_img_instructor-src': 'instructor_img2',
    'accessible_img_instructor-src': 'instructor_img3', 
    'enthusiasm_img_instructor-src': 'instructor_img4',
    'discussion_img_instructor-src': 'instructor_img5', 
    'feedback_img_instructor-src': 'instructor_img6',
    'assignments_img_instructor-src': 'instructor_img7'
}

# We will immediately convert instances of key to value for less messiness
translation = {
    # For overall course questions
    'Course Overall': 'Overall',
    'Workload (hours per week)': 'Workload',
    'Met Gen Ed Goals': 'Goals',
    'Would You Recommend': 'Recommend',
    'Money spent on course': 'Money',

    # Translations for metrics in teacher evaluation
    'Returns Assignments in Timely Fashion': 'Return',
    'Gives Useful Feedback': 'Feedback',
    'Facilitates Discussion & Encourages Participation': 'Participation',
    'Generates Enthusiasm': 'Enthusiasm',
    'Accessible Outside Class': 'Accessible',
    'Effective Lectures or Presentations': 'Lectures',
    'Instructor Overall': 'Overall'
}

# Columns we DO want to analyze.
whitelists = {'course': [r'Overall', r'Materials', r'Assignments', r'Feedback', 
                         r'Section', r'Workload', r'Goals', r'Recommend', r'Money'],
              'instructor': [r'Return', r'Feedback', r'Participation', r'Enthusiasm', 
                             r'Accessible', r'Lectures', r'Overall']}

img_sources = (['course_img' + str(i) for i in range(1, 9)] + 
               ['instructor_img' + str(i) for i in range(1, 8)] +
               ['reason' + str(i) for i in range(1, 8)])

semesters = [('fall', 2011),
             ('spring', 2012),
             ('fall', 2012),
             ('spring', 2013),
             ('fall', 2013),
             ('spring', 2014),
             ('fall', 2014),
             ('spring', 2015),
             ('fall', 2015),
             ('spring', 2016),
             ('fall', 2016),
             ('spring', 2017)]
DATA_PATH = 'data/'

In [28]:
def read_in_raw_data(year, term):
    path = DATA_PATH + term + '_' + str(year) + '.csv'
    return (pd.read_csv(path, na_values=['null'])
            .drop(drop_cols, 1)
            .rename(columns=rename_cols)
            .assign(year=year)
            .assign(term=term))

# Takes a df with two rows describing the same course and merges rows to avoid nans
def fix_duplicate_courses(sel):
    final = sel.iloc[0]
    for i in range(1, len(sel)):
        final = final.fillna(sel.iloc[i])
    return final

# Processes summary stats into lists.
def clean_summary(s):
    if type(s) == type(np.nan):
        return []
    else:
        n = remove_bad_whitespace(s).split('\n')
        return [int(n[1].strip()), int(n[3].strip())]

# Gets int list of histogram for a particular graph.
def clean_img(link):
    if type(link) == type(np.nan):
        return []
    else:
        # Fixes a weird bug where zero becomes blank
        link = (link.replace('--', '-0-')
                .replace('--', '-0-')
                .replace('-.', '-0.')) # Fix 'histobar---1-2-3' like issues
        lst = (link.split('/')[-1:][0]
               .replace('histobar-', '') # For the histogram charts
               .replace('barPercentage-', '') # For the bar charts
               .replace('.png', '')
               .split('-'))
        return [int(x) for x in lst]

def remove_bad_whitespace(s):
    return s.replace('\xc2', '').replace('\xa0', '')

def provide_shorthands(s):
    for k, v in translation.iteritems():
        s = s.replace(k, v)
    return s

def clean_overall(s, whitelist):
    if type(s) == type(np.nan):
        return []
    else:
        s = provide_shorthands(remove_bad_whitespace(s)).replace('\n', '').replace(' ','')
        # Fixes issue with skipping over data if no responses to question
        s = re.sub('(?<=[a-z])0(?=[A-Z])', '00.0', s)
        # Last three characters must be decimal average.  Rest to be split to name, #respondents
        f = lambda x:  re.split('(\d+)', x[:-3])[:2] + [x[-3:]]
        # Serious problems with backslash
        pats = [(whitie + '[0-9]+[^0-9][0-9]') for whitie in whitelist]
        found = [itm for lst in [re.findall(pat, s) for pat in pats] for itm in lst]
        return map(f, found)

def clean_comment(s):
    if type(s) == type(np.nan):
        return []
    else:
        # Engineered to just work.
        filtrated = filter(lambda x: x != '', map(lambda x: x[31:], s.split('\n')[9:-7]))
        return map(lambda x: x.strip(), filtrated)

def clean_instructor(s):
    # First extract first and last name
    if type(s) == type(np.nan) or s == 'No data available':
        return []
    else:
        s = provide_shorthands(s)
        # Some names end with ) or .
        matches = re.findall('\A.*\w\.*\)*(?=Overall)', s)
        if len(matches) == 0:
            raw_names = '","'
        else:
            raw_names = matches[0]
        names = raw_names.replace(' ', '').split(',') # first and last names
        rest = clean_overall(s[len(raw_names):], whitelists['instructor'])
        return [names, rest]
    
# Gets all distribution information and puts it in one dict.
def make_img_dict(row):
    d = {}
    for img_link in img_sources:
        d[img_link] = clean_img(row[img_link])
    row['img_dict'] = d
    return row

def print_debug(d):
    print d
    return d

def add_instructor_info(df):
    df = df.copy()
    # Add instructor first and last name.
    df['instructor_first_name'] = (df['all_instructor']
                                   .apply(lambda x: '' if x == [] else x[0][1]))
    df['instructor_last_name'] = (df['all_instructor']
                                   .apply(lambda x: '' if x == [] else x[0][0]))
    # Simplify the data in all_instructor column
    df['all_instructor'] = (df['all_instructor']
                            .apply(lambda x: [] if x == [] else x[1]))
    return df

def add_number_of_comments(df):
    df = df.copy()
    invalid_cond = df['comments'] == [] or df['comments'] == np.nan
    df['number_of_comments'] = 0 if invalid_cond else len(df['comments'])
    return df

def add_summary_stat_info(df):
    df['enrollment'] = df['summary_stats'].apply(lambda x: [] if x==[] else x[0])
    df['responses'] = df['summary_stats'].apply(lambda x: [] if x==[] else x[1])
    return df.drop('summary_stats', 1)

# Takes column with list of lists in each entry and makes dict in each entry w/same data.
def convert_to_dict(frame, src): # src is either course or instructor
    list_of_lists = frame['all_' + src]
    img_dict = frame['img_dict']
    limit = len(whitelists[src])
    d = {}
    if type(list_of_lists) == type(np.nan):
        return d
    else:
        for idx, lst in enumerate(list_of_lists):
            if len(lst) == 3: # There is some bad data here.  No time to figure it out.
                name = lst[0]
                d[name + '_Respondents'] = int(lst[1])
                d[name + '_Rating'] = float(lst[2])
                # Because some pages have more than 8 things...they get cut off
                if (idx < limit) & (idx != 8): # Without this redundancy it crashes.  Python has bugs.
                    d[name + '_Distribution'] = img_dict[src + '_img' + str(idx + 1)]
        return d
    
def add_focus_columns(df):
    df = df.copy()
    for datum in ['_Rating', '_Respondents', '_Distribution']:
        for col in whitelists['course']:
            df['Course_' + 
               col + datum] = (df['all_course']
                                .apply(lambda d: (d[col + datum] 
                                                  if col + datum in d.keys()
                                                  else np.nan)))
        for col in whitelists['instructor']:
            df['Instructor_' + 
               col + datum] = (df['all_instructor']
                                .apply(lambda d: (d[col + datum] 
                                                  if col + datum in d.keys()
                                                  else np.nan)))
    return df

def sum_careful(x):
    '''Sum a list but return nan if the list was empty (something python should do but doesn't)'''
    return np.nan if (x==[] or type(x) == type(np.nan)) else sum(x)

def separate_course_name_number(df):
    # Get rid of straight duplicates then collect non-nan values to a single column
    df = df.drop_duplicates()
    df = df.groupby('course').apply(fix_duplicate_courses).reset_index(drop=True)
    # Get info on department and course number/title separated.  Difficult because some courses have two depts
    split_colon = pd.DataFrame(df['course'].str.split(':', 1).tolist(), 
                               columns=['to_be_split_again', 'title'])
    # strip removes leading whitespace.
    df.loc[:, 'course_title'] = split_colon['title'].str.strip().values 
    # Splitting in case there are more than one departments offering the course. 
    list_of_lists = split_colon['to_be_split_again'].str.split('/', 1).tolist()
    split_courses = pd.DataFrame([x + [' '] if len(x) < 2 else x for x in list_of_lists], 
                                 columns=['col1', 'col2'])
    split_dept1 = pd.DataFrame(split_courses['col1'].str.split(' ', 1).tolist(), 
                              columns=['department1', 'course_number1'])
    split_dept2 = pd.DataFrame(split_courses['col2'].str.split(' ', 1).tolist(), 
                              columns=['department2', 'course_number2'])
    df.loc[:, 'course_number1'] = split_dept1['course_number1'].values
    df.loc[:, 'department1'] = split_dept1['department1'].values
    df.loc[:, 'course_number2'] = split_dept2['course_number2'].values
    df.loc[:, 'department2'] = split_dept2['department2'].values
    # Make a course identifier being name and department.  Easier database stuff later on.  - RP
    df.loc[:, 'name_key1'] = (df['department1'] + ' ' + df['course_number1']).values
    return df

def fix_workload(df):
    '''Fixes problem whereby workloads of more than 10 hours are parsed incorrectly.  Since we matched
       on d.d we only get one digit by decimal and other goes (incorrectly) to respondents.  This fixes.'''
    df = df.copy()
    condition = ((df['Course_Workload_Distribution'].apply(sum_careful) != 
                  df['Course_Workload_Respondents']))
    df['Course_Workload_Rating'] = np.where(condition, 
                                             (df['Course_Workload_Rating'] + 
                                             (10 * df['Course_Workload_Respondents']
                                              .fillna(0).astype(int).apply(lambda x: x % 10))),
                                             df['Course_Workload_Rating'])
    df['Course_Workload_Respondents'] = np.where(condition, 
                                             (df['Course_Workload_Respondents']
                                              .apply(lambda x: math.floor(x / 10))),
                                             df['Course_Workload_Respondents'])
    return df

In [29]:
def wrangle(term, year):
    print 'Wrangling data for ' + term + ' ' + str(year)
    # Read in raw data and parse course column for alternate courses. 
    df = read_in_raw_data(year, term)
    df = separate_course_name_number(df)

    # Process our complicated columns
    df['all_course'] = df['all_course'].apply(clean_overall, args=(whitelists['course'], ))
    df['all_instructor'] = df['all_instructor'].apply(clean_instructor)
    df['comments'] = df['comments'].apply(clean_comment)
    df['summary_stats'] = df['summary_stats'].apply(clean_summary)

    # Let's simplify things considerably.  No reason for all those columns. 
    df = df.apply(make_img_dict, axis=1).drop(img_sources, 1)

    # Now take consolodated data and put in columns
    df = add_instructor_info(df)
    df = add_summary_stat_info(df)
    df = df.apply(add_number_of_comments, axis=1)
    df['all_course'] = df.apply(convert_to_dict, axis=1, args=('course',))
    df['all_instructor'] = df.apply(convert_to_dict, axis=1, args=('instructor',))
    df = add_focus_columns(df)
    df = fix_workload(df)
    return df

In [30]:
def aggregate_and_trim():
    df = wrangle(*semesters[0])
    for semester in semesters[1:]:
        df = df.append(wrangle(*semester))
    # Parsed all info from these already.
    df = df.drop(['img_dict', 'all_instructor', 'all_course'], 1)
    return df

In [31]:
final = aggregate_and_trim()

Wrangling data for fall 2011
Wrangling data for spring 2012
Wrangling data for fall 2012
Wrangling data for spring 2013
Wrangling data for fall 2013
Wrangling data for spring 2014
Wrangling data for fall 2014
Wrangling data for spring 2015
Wrangling data for fall 2015
Wrangling data for spring 2016
Wrangling data for fall 2016
Wrangling data for spring 2017


In [32]:
final

,course,comments,year,term,course_title,course_number1,department1,course_number2,department2,name_key1,...,Course_Goals_Distribution,Course_Recommend_Distribution,Course_Money_Distribution,Instructor_Return_Distribution,Instructor_Feedback_Distribution,Instructor_Participation_Distribution,Instructor_Enthusiasm_Distribution,Instructor_Accessible_Distribution,Instructor_Lectures_Distribution,Instructor_Overall_Distribution
0,AESTHINT 13: Cultural Agents,[This class is one of the few classes at Harva...,2011,fall,Cultural Agents,13,AESTHINT,,,AESTHINT 13,...,NaN,"[0, 6, 10, 21, 21]",NaN,"[2, 8, 19, 17, 7]","[3, 8, 19, 12, 11]","[3, 10, 13, 8, 7]","[4, 9, 14, 15, 11]","[2, 3, 13, 11, 8]","[2, 1, 3, 6, 6]","[2, 1, 3, 6, 5]"
1,AESTHINT 15: Elements of Rhetoric,"[Substantial work in the form of papers, but v...",2011,fall,Elements of Rhetoric,15,AESTHINT,,,AESTHINT 15,...,NaN,"[0, 2, 6, 9, 25]",NaN,"[0, 2, 4, 17, 19]","[0, 1, 11, 11, 18]","[0, 0, 1, 15, 26]","[0, 0, 6, 9, 27]","[0, 1, 6, 12, 18]","[0, 0, 1, 5, 10]","[0, 0, 1, 4, 9]"
2,AESTHINT 24: First Nights: Five Performance P...,[This class deals with a lot of information-bo...,2011,fall,First Nights: Five Performance Premieres,24,AESTHINT,,,AESTHINT 24,...,NaN,"[2, 3, 26, 35, 41]",NaN,"[0, 0, 8, 27, 73]","[0, 2, 3, 17, 85]","[1, 2, 20, 30, 42]","[0, 1, 4, 12, 91]","[2, 2, 3, 7, 9]","[0, 2, 4, 4, 7]","[0, 2, 3, 5, 6]"
3,"AESTHINT 26: Race, Gender, and Performance","[This is a great, fun, easy, gen-ed that gives...",2011,fall,"Race, Gender, and Performance",26,AESTHINT,,,AESTHINT 26,...,NaN,"[5, 6, 15, 30, 34]",NaN,"[0, 7, 12, 33, 38]","[0, 5, 15, 23, 48]","[1, 3, 8, 22, 45]","[2, 2, 9, 23, 53]","[0, 4, 11, 22, 34]","[0, 3, 4, 7, 7]","[0, 1, 4, 8, 9]"
4,AESTHINT 30: Love In A Dead Language: Classic...,[It's a lot of reading and though there aren't...,2011,fall,Love In A Dead Language: Classical Indian Lite...,30,AESTHINT,,,AESTHINT 30,...,NaN,"[0, 0, 2, 10, 17]",NaN,"[0, 0, 1, 6, 21]","[0, 0, 1, 7, 19]","[0, 0, 1, 3, 21]","[0, 0, 1, 5, 22]","[0, 1, 2, 8, 15]","[0, 1, 1, 4, 6]","[0, 0, 2, 3, 8]"
5,AESTHINT 33: Ancient Fictions: The Ancient No...,"[Finally, a fun way to fulfill the AIU require...",2011,fall,Ancient Fictions: The Ancient Novel in Context,33,AESTHINT,,,AESTHINT 33,...,NaN,"[0, 1, 21, 25, 30]",NaN,"[0, 0, 6, 18, 52]","[0, 0, 6, 12, 59]","[0, 0, 4, 19, 42]","[0, 1, 3, 19, 54]","[0, 0, 6, 9, 32]","[0, 0, 2, 5, 11]","[1, 0, 3, 2, 11]"
6,AESTHINT 35: Forms in Korean Cultural History,[If you take the readings and the assignments ...,2011,fall,Forms in Korean Cultural History,35,AESTHINT,,,AESTHINT 35,...,NaN,"[0, 0, 1, 3, 4]",NaN,"[0, 0, 0, 3, 5]","[0, 0, 2, 3, 3]","[0, 0, 1, 4, 3]","[0, 0, 0, 2, 5]","[0, 0, 1, 4, 3]","[0, 0, 2, 5, 1]","[0, 0, 1, 6, 1]"
7,AESTHINT 37: Introduction to the Bible in the...,[],2011,fall,Introduction to the Bible in the Humanities an...,37,AESTHINT,,,AESTHINT 37,...,NaN,"[25, 29, 53, 31, 31]",NaN,"[12, 26, 40, 45, 47]","[6, 16, 28, 38, 82]","[15, 29, 40, 23, 24]","[6, 20, 29, 32, 83]","[7, 11, 12, 12, 8]","[12, 7, 8, 3, 6]","[6, 7, 7, 8, 6]"
8,AESTHINT 38: The English Language as Literature,[Be prepared to read some potentially dull lit...,2011,fall,The English Language as Literature,38,AESTHINT,,,AESTHINT 38,...,NaN,"[0, 0, 3, 6, 6]",NaN,"[0, 0, 2, 4, 9]","[0, 0, 2, 4, 9]","[0, 0, 3, 6, 3]","[0, 2, 2, 4, 7]","[0, 1, 2, 4, 5]","[0, 0, 1, 2, 2]","[0, 0, 0, 1, 2]"
9,AESTHINT 40: Monuments of Islamic Architecture,[This is an interesting and easy course about ...,2011,fall,Monuments of Islamic Architecture,40,AESTHINT,,,AESTHINT 40,...,NaN,"[0, 0, 4, 5, 3]",NaN,"[1, 0, 3, 5, 3]","[0, 1, 4, 5, 2]","[1, 1, 2, 1, 2]","[0, 1, 4, 2, 5]","[0, 1, 1, 0, 2]","[0, 0, 1, 2, 2]","[0, 0, 0, 3, 2]"


In [33]:
final.to_csv('final.csv')

### Making a list for the website of each class.

In [ ]:
import pandas as pd
import json

In [ ]:
df = pd.read_csv('final.csv')
df['dept_num'] = df['department1'] + ' ' + df['course_number1']
lst = list(set(df['dept_num']))

In [ ]:
with open('course_list.json', 'wb') as outfile:
    json.dump(lst, outfile)

In [ ]:
final

In [2]:
df = pd.read_csv('final.csv')

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (54,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
